In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/JustRAIGS_Train_labels.csv', sep=';')
df.columns

Index(['Eye ID', 'Final Label', 'Fellow Eye ID', 'Age', 'Label G1', 'Label G2',
       'Label G3', 'G1 ANRS', 'G1 ANRI', 'G1 RNFLDS', 'G1 RNFLDI', 'G1 BCLVS',
       'G1 BCLVI', 'G1 NVT', 'G1 DH', 'G1 LD', 'G1 LC', 'G2 ANRS', 'G2 ANRI',
       'G2 RNFLDS', 'G2 RNFLDI', 'G2 BCLVS', 'G2 BCLVI', 'G2 NVT', 'G2 DH',
       'G2 LD', 'G2 LC', 'G3 ANRS', 'G3 ANRI', 'G3 RNFLDS', 'G3 RNFLDI',
       'G3 BCLVS', 'G3 BCLVI', 'G3 NVT', 'G3 DH', 'G3 LD', 'G3 LC'],
      dtype='object')

In [3]:
df.shape

(101423, 37)

In [5]:
def labels(label):
    if label=='NRG': return 0
    elif label=='RG': return 1
    else: return -1

In [6]:
df['Final Label'] = df['Final Label'].map(labels)

In [7]:
[n for n in df.columns if n not in ['Eye ID', 'Final Label']]

['Fellow Eye ID',
 'Age',
 'Label G1',
 'Label G2',
 'Label G3',
 'G1 ANRS',
 'G1 ANRI',
 'G1 RNFLDS',
 'G1 RNFLDI',
 'G1 BCLVS',
 'G1 BCLVI',
 'G1 NVT',
 'G1 DH',
 'G1 LD',
 'G1 LC',
 'G2 ANRS',
 'G2 ANRI',
 'G2 RNFLDS',
 'G2 RNFLDI',
 'G2 BCLVS',
 'G2 BCLVI',
 'G2 NVT',
 'G2 DH',
 'G2 LD',
 'G2 LC',
 'G3 ANRS',
 'G3 ANRI',
 'G3 RNFLDS',
 'G3 RNFLDI',
 'G3 BCLVS',
 'G3 BCLVI',
 'G3 NVT',
 'G3 DH',
 'G3 LD',
 'G3 LC']

# Build datasets for Referrable Glaucoma prediction


In [8]:
df = df.drop([n for n in df.columns if n not in ['Eye ID', 'Final Label']], axis=1)
df.columns

Index(['Eye ID', 'Final Label'], dtype='object')

In [9]:
df=df.rename(columns={'Eye ID': 'image_id', 'Final Label': 'label'})
df.image_id = [n+'.JPG' for n in df.image_id.values]

In [10]:
df.head()

,image_id,label
0,TRAIN000000.JPG,0
1,TRAIN000001.JPG,0
2,TRAIN000002.JPG,0
3,TRAIN000003.JPG,0
4,TRAIN000004.JPG,0


In [11]:
np.unique(df['label'], return_counts=True)

(array([0, 1]), array([98153,  3270]))

In [12]:
num_ims = len(df)
meh, df_val1 = train_test_split(df, test_size=num_ims//5, random_state=0, stratify=df.label)
meh, df_val2 = train_test_split(meh,    test_size=num_ims//5, random_state=0, stratify=meh.label)
meh, df_val3 = train_test_split(meh,    test_size=num_ims//5, random_state=0, stratify=meh.label)
df_val5, df_val4 = train_test_split(meh,test_size=num_ims//5, random_state=0, stratify=meh.label)

df_train1 = pd.concat([df_val2,df_val3,df_val4,df_val5], axis=0)
df_train2 = pd.concat([df_val1,df_val3,df_val4,df_val5], axis=0)
df_train3 = pd.concat([df_val1,df_val2,df_val4,df_val5], axis=0)
df_train4 = pd.concat([df_val1,df_val2,df_val3,df_val5], axis=0)
df_train5 = pd.concat([df_val1,df_val2,df_val3,df_val4], axis=0)

df_train1.to_csv('data/tr_rg_f1.csv', index=None)
df_val1.to_csv('data/vl_rg_f1.csv', index=None)

df_train2.to_csv('data/tr_rg_f2.csv', index=None)
df_val2.to_csv('data/vl_rg_f2.csv', index=None)

df_train3.to_csv('data/tr_rg_f3.csv', index=None)
df_val3.to_csv('data/vl_rg_f3.csv', index=None)

df_train4.to_csv('data/tr_rg_f4.csv', index=None)
df_val4.to_csv('data/vl_rg_f4.csv', index=None)

df_train5.to_csv('data/tr_rg_f5.csv', index=None)
df_val5.to_csv('data/vl_rg_f5.csv', index=None)

# Adding uncertainty
If G1 == G2, that is final. If G1 != G2, then G3 says something. Instead of using 'Final Label' directly, it would be maybe better to give it a label of (1/4, 3/4) or (3/4, 1/4) in cases where one grader disagreed, right?

It's not that easy. It turns out that:


``We evaluated the performance of the graders periodically and occasionally had to remove a grader. In such a case, all labels were removed because they could not be trusted.``

``In general, these eyes were re-evaluated by other graders - but not if they were already reviewed by the third grader. After all, the third grader overruled the other graders, so the labels of graders 1 and 2 were no longer relevant.``

``The 'U' means Ungradable, which could be selected by graders if they thought the image's quality was too poor to judge the eye. We excluded all image with a final Ungradable label in this challenge, but you can still come across them for the graders.``


In [184]:
df = pd.read_csv('data/JustRAIGS_Train_labels.csv', sep=';', index_col=False)
df.columns

Index(['Eye ID', 'Final Label', 'Fellow Eye ID', 'Age', 'Label G1', 'Label G2',
       'Label G3', 'G1 ANRS', 'G1 ANRI', 'G1 RNFLDS', 'G1 RNFLDI', 'G1 BCLVS',
       'G1 BCLVI', 'G1 NVT', 'G1 DH', 'G1 LD', 'G1 LC', 'G2 ANRS', 'G2 ANRI',
       'G2 RNFLDS', 'G2 RNFLDI', 'G2 BCLVS', 'G2 BCLVI', 'G2 NVT', 'G2 DH',
       'G2 LD', 'G2 LC', 'G3 ANRS', 'G3 ANRI', 'G3 RNFLDS', 'G3 RNFLDI',
       'G3 BCLVS', 'G3 BCLVI', 'G3 NVT', 'G3 DH', 'G3 LD', 'G3 LC'],
      dtype='object')

In [185]:
df = df.drop([n for n in df.columns if n not in ['Eye ID', 'Final Label', 'Label G1', 'Label G2','Label G3']], 
             axis=1)
df.columns

Index(['Eye ID', 'Final Label', 'Label G1', 'Label G2', 'Label G3'], dtype='object')

In [186]:
def labels(label):
    if label=='NRG': return 0
    elif label=='RG': return 1
    elif label=='U': return 2
    elif label==np.nan: return 3
df['Final Label'].unique(), df['Label G1'].unique(), df['Label G2'].unique()

(array(['NRG', 'RG'], dtype=object),
 array(['NRG', 'RG', 'U', nan], dtype=object),
 array(['NRG', nan, 'RG', 'U'], dtype=object))

In [187]:
df['Final Label'] = df['Final Label'].map(labels)
df['Label G1'] = df['Label G1'].map(labels)
df['Label G2'] = df['Label G2'].map(labels)
df['Label G3'] = df['Label G3'].map(labels)

df['Final Label'].unique(), df['Label G1'].unique(), df['Label G2'].unique()

In [188]:
df['Final Label'].unique(), df['Label G1'].unique(), df['Label G2'].unique()

(array([0, 1]), array([ 0.,  1.,  2., nan]), array([ 0., nan,  1.,  2.]))

So here is what we are gonna do: 

* If any grader said 'U', we assign a label of 0.1 if Final Grade==0, and of 0.9 if Final Grade==1
* For images where Label G1 is nan, if Label G2 != Label G3, we assign a 0.8 if G3 says 1, and 0.2 if G3 says 0
* Same for images where Label G2 is nan
* For images where Label G1 != Label G2, 
    - if Label G1==Label G3==1, then 0.85
    - if Label G1==Label G3==0, then 0.15
    - if Label G2==Label G3==1, then 0.85
    - if Label G2==Label G3==0, then 0.15
* And that is all, right?

Note that this needs to happen only for training dfs, not for validation dfs, which require hard labels and we will be using Final Label in that case.

In [217]:
def map_label(label, low=0, high=1):
    if label == 0: return low
    elif label==1: return high
    else: return label


def soften_labels(df):
    df.loc[(df['Label G1']==2) | (df['Label G2']==2), 'Final Label']=\
df.loc[(df['Label G1']==2) | (df['Label G2']==2), 'Final Label'].apply(lambda x:map_label(x, low=0.1, high=0.9))
    df.loc[(np.isnan(df['Label G2'])) & (~np.isnan(df['Label G1'])) & (df['Label G1']!=df['Label G3']), 'Final Label']=\
df.loc[(np.isnan(df['Label G2'])) & (~np.isnan(df['Label G1'])) & (df['Label G1']!=df['Label G3']), 'Final Label'].apply(lambda x:map_label(x, low=0.15, high=0.85))
    return df
#     print(ambiguous.shape)
    #return this_df

In [190]:
df.shape

(101423, 5)

In [209]:
df['Label G1'].isna().sum()

274

In [210]:
df['Label G2'].isna().sum()

1377

All samples that are nan for G1 are also nan for G2

In [211]:
df.loc[(np.isnan(df['Label G1'])) & (np.isnan(df['Label G2'])), 'Final Label'].shape

(274,)

Therefore no samples have nan in G1 and not nan in G2:

In [212]:
df.loc[(np.isnan(df['Label G1'])) & (~np.isnan(df['Label G2'])), 'Final Label'].shape

(0,)

And there are samples that are nans for G2 and not for G1

In [213]:
df.loc[(np.isnan(df['Label G2'])) & (~np.isnan(df['Label G1'])), 'Final Label'].shape

(1103,)

For these, if G1 is different than G3, we soften the label:

In [214]:
df.loc[(np.isnan(df['Label G2'])) & (~np.isnan(df['Label G1'])) & (df['Label G1']!=df['Label G3']), 'Final Label'].shape

(343,)

In [216]:
df.loc[(np.isnan(df['Label G2'])) & (~np.isnan(df['Label G1'])) & (df['Label G1']!=df['Label G3']), 'Final Label']=df.loc[(np.isnan(df['Label G2'])) & (~np.isnan(df['Label G1'])) & (df['Label G1']!=df['Label G3']), 'Final Label'].apply(lambda x:map_label(x, low=0.1, high=0.9))

# Done

In [237]:
def map_label(label, low=0, high=1):
    if label == 0: return low
    elif label==1: return high
    else: return label


def soften_labels(df):
    df.loc[(df['Label G1']==2) | (df['Label G2']==2), 'label']=\
df.loc[(df['Label G1']==2) | (df['Label G2']==2), 'label'].apply(lambda x:map_label(x, low=0.1, high=0.9))
    df.loc[(np.isnan(df['Label G2'])) & (~np.isnan(df['Label G1'])) & (df['Label G1']!=df['Label G3']), 'label']=\
df.loc[(np.isnan(df['Label G2'])) & (~np.isnan(df['Label G1'])) & (df['Label G1']!=df['Label G3']), 'label'].apply(lambda x:map_label(x, low=0.15, high=0.85))
    return df
#     print(ambiguous.shape)
    #return this_df

In [238]:
df = pd.read_csv('data/JustRAIGS_Train_labels.csv', sep=';', index_col=False)
df=df.rename(columns={'Eye ID': 'image_id', 'Final Label': 'label'})
df.image_id = [n+'.JPG' for n in df.image_id.values]
df = df.drop([n for n in df.columns if n not in ['image_id','label','Label G1','Label G2','Label G3']],axis=1)

In [239]:
df['label'] = df['label'].map(labels)
df['Label G1'] = df['Label G1'].map(labels)
df['Label G2'] = df['Label G2'].map(labels)
df['Label G3'] = df['Label G3'].map(labels)

df['label'].unique(), df['Label G1'].unique(), df['Label G2'].unique()

(array([0, 1]), array([ 0.,  1.,  2., nan]), array([ 0., nan,  1.,  2.]))

In [240]:
df = soften_labels(df)

In [241]:
df['label'].unique(), df['Label G1'].unique(), df['Label G2'].unique()

(array([0.  , 1.  , 0.1 , 0.9 , 0.15, 0.85]),
 array([ 0.,  1.,  2., nan]),
 array([ 0., nan,  1.,  2.]))